In [1]:
from ARB_Volume_Equations import *

In [2]:
# Creates a class that holds the equations and related attributes of equations to be used for each species.

class Spp_eqs:
    
    def __init__(self, common_name = None, wood_type = None, WOR = None, WWA = None, EOR = None, EWA = None, CA = None):
        '''
        Instantiates the class containining information about a tree species with these inputs:
        common_name: common name of the species (string)
        wood_type: hardwood or softwood, entered as 'HW' or 'SW' (string)
        WOR, WWA, EOR, EQ, CA: the equation number (or none) to be used for that species in that region
        '''
        self.name = common_name
        self.wood_type = wood_type
        
        self.WOR = eval("Eq_"+str(WOR))
        self.WWA = eval("Eq_"+str(WWA))
        self.EOR = eval("Eq_"+str(EOR))
        self.EWA = eval("Eq_"+str(EWA))
        self.CA = eval("Eq_"+str(CA))

In [5]:
# Creates all the classes for all species covered in the ARB publications for Oregon, Washington, and California
# Assigns equations numbers to each species by region

# SOFTWOODS
SF = Spp_eqs("Pacific silver fir", "SW", 11, 11, 10, 10, 11)
BF = Spp_eqs("Bristlecone fir", "SW", None, None, None, None, 18)
WF = Spp_eqs("White fir", "SW", 23, 11, 10, 10, 23)
GF = Spp_eqs("Grand fir", "SW", 11, 11, 10, 10, 23)
AF = Spp_eqs("Subalpine fir", "SW", 11, 11, 10, 10, 18)
RF = Spp_eqs("California red fir", "SW", 18, None, 10, None, 18)
SH = Spp_eqs("Shasta red fir", "SW", 18, None, 18, None, 18)
NF = Spp_eqs("Noble fir", "SW", 11, 11, 10, 10, 18)
PC = Spp_eqs("Port-Orford-cedar", "SW", 19, 19, 19, 19, 8)
YC = Spp_eqs("Alaska-cedar", "SW", 9, 9, 8, 8, 8)
# All cypress species (Arizona, Baker, Monterey, Sargent's and McNabb Cypress) consolidated into one class
CY = Spp_eqs("Cypress", "SW", None, None, None, None, 19)
CJ = Spp_eqs("California juniper", "SW", None, None, None, None, 19)
WJ = Spp_eqs("Western juniper", "SW", 21, 21, 21, 21, 21)
UJ = Spp_eqs("Utah juniper", "SW", None, None, None, None, 14)
RJ = Spp_eqs("Rocky mountain juniper", "SW", None, None, 142, None, 142) # 3rd and 5th eqn are 14.2
LL = Spp_eqs("Subalpine larch", "SW", None, 22, 22, 22, None)
WL = Spp_eqs("Western larch", "SW", 22, 22, 22, 22, 22)
IC = Spp_eqs("Incense-cedar", "SW", 19, 19, 19, 19, 19)
BR = Spp_eqs("Brewer spruce", "SW", 13, None, 13, 13, 12)
ES = Spp_eqs("Engelmann spruce", "SW", 13, 13, 12, 12, 12)
SS = Spp_eqs("Sitka spruce", "SW", 13, 13, 13, 13, 12)
WB = Spp_eqs("Whitebark pine", "SW", 15, 15, 15, 15, 20)
BC = Spp_eqs("Bristlecone pine", "SW", None, None, None, None, 16)
KP = Spp_eqs("Knobcone pine", "SW", 15, 15, 15, None, 16)
FP = Spp_eqs("Foxtail pine", "SW", None, None, None, None, 16)
LP = Spp_eqs("Lodgepole pine", "SW", 15, 15, 15, 15, 16)
CP = Spp_eqs("Coulter pine", "SW", None, None, None, None, 5)
LM = Spp_eqs("Limber pine", "SW", None, None, 16, None, 16)
JP = Spp_eqs("Jeffrey pine", "SW", 5, None, 4, None, 5)
SP = Spp_eqs("Sugar pine", "SW", 20, None, 20, None, 20)
WP = Spp_eqs("Western white pine", "SW", 15, 15, 15, 15, 20)
BP = Spp_eqs("Bishop pine", "SW", 16, None, 16, None, 16)
PP = Spp_eqs("Ponderosa pine", "SW", 5, 4, 4, 4, 5)
MP = Spp_eqs("Monterey pine", "SW", None, None, None, None, 16)
GP = Spp_eqs("Gray pine", "SW", None, None, None, None, 5)
ST = Spp_eqs("Scotch pine", "SW", 15, None, None, None, None)
PM = Spp_eqs("Singleleaf pinyon pine", "SW", None, None, None, None, 141) # last eqn is 14.1
WE = Spp_eqs("Washoe pine", "SW", None, None, None, None, 5)
GB = Spp_eqs("Great Basin bristlecone pine", "SW", None, None, None, None, 16)
BD = Spp_eqs("Bigcone Douglas-fir", "SW", None, None, None, None, 3)
DF = Spp_eqs("Douglas-fir", "SW", 1, 1, 2, 2, 3)
RW = Spp_eqs("Redwood", "SW", 24, 24, 24, None, 24)
GS = Spp_eqs("Giant Sequoia", "SW", 24, None, 24, None, 24)
PY = Spp_eqs("Pacific yew", "SW", 9, 9, 8, 8, 8)
RC = Spp_eqs("Western redcedar", "SW", 9, 9, 8, 8, 8)
CN = Spp_eqs("California nutmeg", "SW", None, None, None, None, 8)
WH = Spp_eqs("Western hemlock", "SW", 6, 6, 6, 6, 6)
MH = Spp_eqs("Mountain hemlock", "SW", 17, 17, 17, 17, 17)
XC = Spp_eqs("Unknown conifer", "SW", 17, 17, 17, 17, 17)